In [18]:
DATA_DIR = '/Users/pujanmaharjan/uni adelaide/uofa_research_project/datasets'
import pandas as pd
import os
from contextlib import contextmanager
import time
import numpy as np
from joblib import delayed, Parallel

def load_stock_data(stock_id: int, directory: str) -> pd.DataFrame:
    return pd.read_parquet(os.path.join(DATA_DIR, 'optiver-realized-volatility-prediction', directory, f'stock_id={stock_id}'))

from enum import Enum
class DataBlock(Enum):
    TRAIN = 1
    TEST = 2
    BOTH = 3

def load_data(stock_id: int, stem: str, block: DataBlock) -> pd.DataFrame:
    if block == DataBlock.TRAIN:
        return load_stock_data(stock_id, f'{stem}_train.parquet')
    elif block == DataBlock.TEST:
        return load_stock_data(stock_id, f'{stem}_test.parquet')
    else:
        return pd.concat([
            load_data(stock_id, stem, DataBlock.TRAIN),
            load_data(stock_id, stem, DataBlock.TEST)
        ]).reset_index(drop=True)
    
def load_book(stock_id: int, block: DataBlock=DataBlock.TRAIN) -> pd.DataFrame:
    return load_data(stock_id, 'book', block)

def load_trade(stock_id: int, block=DataBlock.TRAIN) -> pd.DataFrame:
    return load_data(stock_id, 'trade', block)

def calc_wap1(df: pd.DataFrame) -> pd.Series:
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1']) / (df['bid_size1'] + df['ask_size1'])
    return wap

def calc_wap2(df: pd.DataFrame) -> pd.Series:
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2']) / (df['bid_size2'] + df['ask_size2'])
    return wap

def realized_volatility(series):
    return np.sqrt(np.sum(series**2))

def log_return(series: np.ndarray):
    return np.log(series).diff()

def log_return_df2(series: np.ndarray):
    return np.log(series).diff(2)

def flatten_name(prefix, src_names):
    ret = []
    for c in src_names:
        if c[0] in ['time_id', 'stock_id']:
            ret.append(c[0])
        else:
            ret.append('_'.join([prefix] + list(c)))
    return ret

def make_book_feature(stock_id, block):
    book = load_book(stock_id, block)
    book['wap1'] = calc_wap1(book)
    book['wap2'] = calc_wap2(book)
    book['log_return1'] = book.groupby(['time_id'], group_keys=False)['wap1'].apply(log_return)
    book['log_return2'] = book.groupby(['time_id'], group_keys=False)['wap2'].apply(log_return)
    book['log_return_ask1'] = book.groupby(['time_id'], group_keys=False)['ask_price1'].apply(log_return)
    book['log_return_ask2'] = book.groupby(['time_id'], group_keys=False)['ask_price2'].apply(log_return)
    book['log_return_bid1'] = book.groupby(['time_id'], group_keys=False)['bid_price1'].apply(log_return)
    book['log_return_bid2'] = book.groupby(['time_id'], group_keys=False)['bid_price2'].apply(log_return)

    features = {
        'wap1': [np.sum],
        'wap2': [np.sum],
        'log_return1': [realized_volatility],
        'log_return2': [realized_volatility],
        'log_return_ask1': [realized_volatility],
        'log_return_ask2': [realized_volatility],
        'log_return_bid1': [realized_volatility],
        'log_return_bid2': [realized_volatility],
    }

    agg = book.groupby('time_id', group_keys=False).agg(features).reset_index(drop=False)
    agg.columns = flatten_name('book', agg.columns)
    agg['stock_id'] = stock_id
    return agg

def make_trade_feature(stock_id, block):
    trade = load_trade(stock_id, block)
    trade['log_return'] = trade.groupby('time_id', group_keys=False)['price'].apply(log_return)

    features = {
        'log_return':[realized_volatility],
        'seconds_in_bucket':['count'],
        'size':[np.sum],
        'order_count':[np.mean],
    }

    agg = trade.groupby('time_id', group_keys=False).agg(features).reset_index()
    agg.columns = flatten_name('book', agg.columns)
    agg['stock_id'] = stock_id
    return agg

@contextmanager
def timer(name):
    s = time.time()
    yield
    e = time.time() - s
    print(f"[{name}] {e:.3f}sec")

def make_rv_lags(df_rv_lag, number_of_periods: int):
    # add past rv
    for i in range(1, number_of_periods + 1):
        df_rv_lag[f'lag_{i}_rv'] = df_rv_lag['target'].shift(periods=i)

    df_rv_lag = df_rv_lag.dropna()
    return df_rv_lag

def make_features(base, block):
    stock_ids = set(base['stock_id'])
    with timer('books'):
        books = Parallel(n_jobs=-1)(delayed(make_book_feature)(stock_id, block) for stock_id in stock_ids)
        book = pd.concat(books)

    with timer('trades'):
        trades = Parallel(n_jobs=-1)(delayed(make_trade_feature)(stock_id, block) for stock_id in stock_ids)
        trade = pd.concat(trades)

    with timer('extra features'):
        df = pd.merge(base, book, on=['stock_id', 'time_id'], how='left')
        df = pd.merge(df, trade, on=['stock_id', 'time_id'], how='left')

    df = make_rv_lags(df, 1)
    
    return df

def get_all_train_data():
    train = pd.read_csv(os.path.join(DATA_DIR, 'optiver-realized-volatility-prediction', 'train_time_id_ordered.csv'))
    return train

def get_unique_stock_ids():
    train = get_all_train_data()
    return list(train['stock_id'].unique())

def get_train_data(stock_ids_to_include):
    train = get_all_train_data()
    print('Train.shape all ', train.shape)
    train = train[train['stock_id'].isin(stock_ids_to_include)]
    train = train.reset_index(drop=True)
    print('Train.shape sample ', train.shape)
    return train

def create_rv(stock_ids, outputfile_name):
    train_df = get_train_data(stock_ids)
    df_features = make_features(train_df, DataBlock.TRAIN)
    df_features.to_csv(os.path.join("./data/", outputfile_name), index=False)
    return df_features


In [21]:
stock_ids = [0,1]
rv_df = create_rv(stock_ids, 'stock_data_basic_features_stock_0.csv')

Train.shape all  (428932, 4)
Train.shape sample  (7660, 4)
[books] 5.401sec
[trades] 0.883sec
[extra features] 0.009sec


In [22]:
rv_df

,stock_id,time_id,target,time_id_order,book_wap1_sum,book_wap2_sum,book_log_return1_realized_volatility,book_log_return2_realized_volatility,book_log_return_ask1_realized_volatility,book_log_return_ask2_realized_volatility,book_log_return_bid1_realized_volatility,book_log_return_bid2_realized_volatility,book_log_return_realized_volatility,book_seconds_in_bucket_count,book_size_sum,book_order_count_mean,lag_1_rv
1,0,4294,0.003267,0,185.004046,184.996116,0.007026,0.010722,0.004446,0.004511,0.004145,0.009287,0.003655,14,2034,3.857143,0.006736
2,0,24033,0.002580,1,340.516826,340.519624,0.004136,0.005465,0.002225,0.002461,0.002483,0.002859,0.001459,26,1755,1.807692,0.003267
3,1,24033,0.003553,1,324.567627,324.582397,0.004334,0.005162,0.003178,0.003832,0.002939,0.002530,0.002521,37,3756,2.729730,0.002580
4,1,5666,0.002267,2,369.722900,369.729675,0.003629,0.004770,0.002739,0.003192,0.002457,0.002641,0.001978,55,7704,2.600000,0.003553
5,0,5666,0.002051,2,294.704533,294.708897,0.002395,0.003925,0.001526,0.002125,0.001539,0.002011,0.000801,29,1313,1.965517,0.002267
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7655,1,15365,0.005306,3827,563.827393,563.809998,0.006208,0.007523,0.003681,0.003652,0.003593,0.003435,0.004191,93,9316,3.913978,0.002017
7656,0,10890,0.003475,3828,282.783980,282.782871,0.003142,0.004277,0.001730,0.001558,0.001662,0.001805,0.001697,39,5954,4.384615,0.005306
7657,1,10890,0.005982,3828,557.041626,557.046875,0.004059,0.005246,0.002556,0.002579,0.003326,0.003293,0.002679,102,16280,4.176471,0.003475
7658,0,29316,0.002846,3829,193.189315,193.192407,0.002736,0.004340,0.001573,0.001381,0.001685,0.002205,0.001302,31,3082,4.129032,0.005982


In [33]:
rv_df = rv_df[rv_df['time_id_order'] != 0]
rv_df

,stock_id,time_id,target,time_id_order,book_wap1_sum,book_wap2_sum,book_log_return1_realized_volatility,book_log_return2_realized_volatility,book_log_return_ask1_realized_volatility,book_log_return_ask2_realized_volatility,book_log_return_bid1_realized_volatility,book_log_return_bid2_realized_volatility,book_log_return_realized_volatility,book_seconds_in_bucket_count,book_size_sum,book_order_count_mean,lag_1_rv
2,0,24033,0.002580,1,340.516826,340.519624,0.004136,0.005465,0.002225,0.002461,0.002483,0.002859,0.001459,26,1755,1.807692,0.003267
3,1,24033,0.003553,1,324.567627,324.582397,0.004334,0.005162,0.003178,0.003832,0.002939,0.002530,0.002521,37,3756,2.729730,0.002580
4,1,5666,0.002267,2,369.722900,369.729675,0.003629,0.004770,0.002739,0.003192,0.002457,0.002641,0.001978,55,7704,2.600000,0.003553
5,0,5666,0.002051,2,294.704533,294.708897,0.002395,0.003925,0.001526,0.002125,0.001539,0.002011,0.000801,29,1313,1.965517,0.002267
6,0,29740,0.002364,3,206.123983,206.132851,0.001790,0.003601,0.001658,0.001665,0.001442,0.001747,0.000861,26,1701,1.807692,0.002051
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7655,1,15365,0.005306,3827,563.827393,563.809998,0.006208,0.007523,0.003681,0.003652,0.003593,0.003435,0.004191,93,9316,3.913978,0.002017
7656,0,10890,0.003475,3828,282.783980,282.782871,0.003142,0.004277,0.001730,0.001558,0.001662,0.001805,0.001697,39,5954,4.384615,0.005306
7657,1,10890,0.005982,3828,557.041626,557.046875,0.004059,0.005246,0.002556,0.002579,0.003326,0.003293,0.002679,102,16280,4.176471,0.003475
7658,0,29316,0.002846,3829,193.189315,193.192407,0.002736,0.004340,0.001573,0.001381,0.001685,0.002205,0.001302,31,3082,4.129032,0.005982


In [34]:
rv_df.columns

Index(['stock_id', 'time_id', 'target', 'time_id_order', 'book_wap1_sum',
       'book_wap2_sum', 'book_log_return1_realized_volatility',
       'book_log_return2_realized_volatility',
       'book_log_return_ask1_realized_volatility',
       'book_log_return_ask2_realized_volatility',
       'book_log_return_bid1_realized_volatility',
       'book_log_return_bid2_realized_volatility',
       'book_log_return_realized_volatility', 'book_seconds_in_bucket_count',
       'book_size_sum', 'book_order_count_mean', 'lag_1_rv'],
      dtype='object')

In [47]:
def save_numpy(data, file_path):
    file_path_to_save = os.path.join("./data/", file_path)
    with open(file_path_to_save, 'wb') as f:
        np.save(f, data)

def read_numpy(file_path):
    with open(file_path, 'rb') as f:
        return np.load(f)

def create_stock_data(data_df, window_len, file_path_to_save):
    time_ids = list(data_df['time_id'].unique())
    print('time_ids ', time_ids)
    stock_ids = list(data_df['stock_id'].unique())
    print('stock_ids ', stock_ids)
    # features = data_df.columns
    features = [x for x in data_df.columns if x not in ['time_id', 'stock_id']]
    print('features ', features)
    all_data = []
    time_id_counter = 0
    time_id_batch = []
    for time_id in time_ids:
        time_id_counter += 1
        stock_level_data = []
        for stock_id in stock_ids:            
            row_data = data_df[(data_df['time_id'] == time_id) & (data_df['stock_id'] == stock_id)]
            if len(row_data) == 0:
                print('row_data not found for time_id ', time_id, ' and stock_id ', stock_id)
                continue
            # print('row_data ', row_data)
            features_data = row_data[features]
            # print('features data ', features_data)
            stock_level_data.append(list(features_data.values[0]))

        time_id_batch.append(stock_level_data)

        if time_id_counter == window_len:
            all_data.append(time_id_batch)
            time_id_batch = []
            time_id_counter = 0
            
    all_data_np = np.array(all_data)
    # print('all_data shape ', all_data_np.shape)

    save_numpy(all_data_np, file_path_to_save)
    
    return all_data_np

def create_multidimensional_input_data(stock_ids, features_file_path, stock_data_file_path):
    rv_df = create_rv(stock_ids, features_file_path)
    rv_df = rv_df[rv_df['time_id_order'] != 0]
    stock_4d = create_stock_data(rv_df, 1, stock_data_file_path)
    return stock_4d

In [48]:
stock_ids_similar_rv = [0,7,11,16]
stock_4d_similar_rv = create_multidimensional_input_data(stock_ids_similar_rv,'stock_data_basic_features_stock_similar_rv.csv', 'stock_multi_dimensional_similar_rv.npy')
print('stock_4d_similar_rv shape ', stock_4d_similar_rv.shape)

Train.shape all  (428932, 4)
Train.shape sample  (15320, 4)
[books] 4.413sec
[trades] 0.712sec
[extra features] 0.009sec
time_ids  [24033, 5666, 29740, 22178, 31984, 13217, 31570, 20351, 13421, 13473, 12011, 16992, 9077, 20135, 250, 9664, 9166, 4543, 12523, 31173, 18940, 27071, 17914, 16454, 28993, 4743, 24661, 7818, 29761, 6619, 26308, 31814, 28422, 27857, 6723, 1587, 17387, 16869, 18498, 10480, 25716, 12782, 4377, 4173, 2139, 23054, 5032, 21695, 9021, 454, 3465, 19579, 24334, 4958, 19678, 9583, 24010, 229, 10349, 20631, 2891, 22616, 27406, 14205, 20581, 19810, 4310, 8615, 17825, 3008, 25772, 554, 10220, 14769, 18949, 10682, 15352, 14908, 5497, 26937, 10230, 27467, 7447, 19554, 25680, 13491, 15846, 11886, 7677, 4927, 12957, 8426, 18020, 7854, 22487, 21328, 25163, 26024, 1822, 14977, 2075, 19090, 13124, 30986, 26021, 10070, 29120, 18445, 20196, 2600, 12486, 10938, 11809, 18628, 25570, 2591, 5793, 30512, 17264, 5206, 26450, 6566, 20935, 4031, 3046, 20453, 19495, 5831, 14070, 26903, 2021

In [49]:
stock_ids_dissimilar_rv = [0,3,9,10]
stock_4d_dissimilar_rv = create_multidimensional_input_data(stock_ids_dissimilar_rv,'stock_data_basic_features_stock_dissimilar_rv.csv', 'stock_multi_dimensional_dissimilar_rv.npy')
print('stock_4d_dissimilar_rv shape ', stock_4d_dissimilar_rv.shape)

Train.shape all  (428932, 4)
Train.shape sample  (15320, 4)
[books] 4.722sec
[trades] 0.662sec
[extra features] 0.006sec
time_ids  [24033, 5666, 29740, 22178, 31984, 13217, 31570, 20351, 13421, 13473, 12011, 16992, 9077, 20135, 250, 9664, 9166, 4543, 12523, 31173, 18940, 27071, 17914, 16454, 28993, 4743, 24661, 7818, 29761, 6619, 26308, 31814, 28422, 27857, 6723, 1587, 17387, 16869, 18498, 10480, 25716, 12782, 4377, 4173, 2139, 23054, 5032, 21695, 9021, 454, 3465, 19579, 24334, 4958, 19678, 9583, 24010, 229, 10349, 20631, 2891, 22616, 27406, 14205, 20581, 19810, 4310, 8615, 17825, 3008, 25772, 554, 10220, 14769, 18949, 10682, 15352, 14908, 5497, 26937, 10230, 27467, 7447, 19554, 25680, 13491, 15846, 11886, 7677, 4927, 12957, 8426, 18020, 7854, 22487, 21328, 25163, 26024, 1822, 14977, 2075, 19090, 13124, 30986, 26021, 10070, 29120, 18445, 20196, 2600, 12486, 10938, 11809, 18628, 25570, 2591, 5793, 30512, 17264, 5206, 26450, 6566, 20935, 4031, 3046, 20453, 19495, 5831, 14070, 26903, 2021